In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Meyer\Documents\Dom_Detecter_Code\Dom_Detecter\dom_detecter_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
directory = r'C:\Users\Meyer\Documents\Dom_Detecter\UNSW_csv_27_28'
all_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

In [4]:
list_of_dataframes = [pd.read_csv(os.path.join(directory, f)) for f in all_files]
merged_dataframe = pd.concat(list_of_dataframes, ignore_index=True)

C:\Users\Meyer\AppData\Local\Temp\ipykernel_26092\2217386177.py:1: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  list_of_dataframes = [pd.read_csv(os.path.join(directory, f)) for f in all_files]


In [5]:
# Read the mapping DataFrame from Excel
mapping_df = pd.read_excel(r'C:\Users\Meyer\Documents\Dom_Detecter\UNSW_csv_23-02\mapping\mapping.xlsx', engine='openpyxl')

In [6]:
df_merged = merged_dataframe

In [7]:
# Create a mapping dictionary
mapping_dict = dict(zip(mapping_df.packet_eth_src, mapping_df.target))

In [8]:
# Apply the mapping to the 'packet_eth_src' column in df_merged
df_merged['target'] = df_merged['packet_eth_src'].map(mapping_dict)

# Fill any NaN values with 'unknown'
df_merged['target'] = df_merged['target'].fillna('unknown')

In [9]:
df_merged.tail()

,idx,protocol,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,...,End_Time,Duration,Flow_Volume,Flow_Rate,Inter_Arrival_Time,Total_Fwd_Packets,Total_Bwd_Packets,Fwd_Packet_Length_Total,Bwd_Packet_Length_Total,target
2546030,2018996,TCP,192.168.1.166,58387.0,195.154.50.167,1935.0,66.0,ack,11632.0,64.0,...,1.475071e+09,6999.330991,187002,26.717125,1.424092,1371,1369,93567,93435,unknown
2546031,2018997,TCP,195.154.50.167,1935.0,192.168.1.166,58387.0,66.0,ack,34816.0,47.0,...,1.475071e+09,6999.330991,187002,26.717125,0.293056,1371,1369,93567,93435,unknown
2546032,2018998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,0.000000,0,0.000000,1.062628,0,0,0,0,unknown
2546033,2018999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,0.000000,0,0.000000,0.001853,0,0,0,0,unknown
2546034,2019000,TCP,192.168.1.106,47940.0,52.87.241.159,443.0,156.0,ack,20392.0,64.0,...,1.475071e+09,7086.521486,1983849,279.946798,0.823423,6672,6572,1530795,453054,unknown


In [10]:
# Count the number of rows
num_rows = df_merged.shape[0]

# Print the number of rows
print(f"The DataFrame has {num_rows} rows.")

The DataFrame has 2546035 rows.


In [11]:
# Count the number of packets for each target and print it
target_counts = df_merged['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown               2159243
samsung_cam             86763
amazon_echo             65372
tp-link_cam             51581
netatmo_cam             49392
belkin_motion           42694
smart_things            24487
belkin_switch           20292
withings_sleep          11237
triby_speaker            8580
netatmo_weather          7325
pixstar_photoframe       7175
hp_printer               4354
lifx                     2975
tp-link_plug             2270
ihome_plug               1760
withings_scale            286
nest_smoke                195
blipcare_bp_meter          54
Name: count, dtype: int64


In [12]:
# Group by device identifier and count unique flows
unique_flows_per_device = df_merged.groupby('target')['Flow'].nunique()

# Display the counts
print(unique_flows_per_device)

target
amazon_echo            4377
belkin_motion          2789
belkin_switch          1189
blipcare_bp_meter         4
hp_printer               22
ihome_plug               16
lifx                    449
nest_smoke               12
netatmo_cam            1375
netatmo_weather         250
pixstar_photoframe     1756
samsung_cam            6081
smart_things            619
tp-link_cam             423
tp-link_plug            178
triby_speaker            49
unknown               97110
withings_scale            4
withings_sleep          322
Name: Flow, dtype: int64


In [13]:
# Sum up all the unique flows
total_unique_flows = unique_flows_per_device.sum()

# Display the total count of unique flows
print(f"Total unique flows: {total_unique_flows}")

Total unique flows: 117025


In [15]:
df = df_merged

In [16]:
# Step 1: Filter out targets with less than 10,000 packets
targets_to_keep = df['target'].value_counts()
targets_to_keep = targets_to_keep[targets_to_keep >= 5000].index.tolist()
filtered_df = df[df['target'].isin(targets_to_keep)]

In [17]:
# Step 2: Sample 10,000 packets for each target
sampled_df = pd.DataFrame()
for target in filtered_df['target'].unique():
    sampled_df = pd.concat([sampled_df, filtered_df[filtered_df['target'] == target].sample(n=5000, random_state=1)])

# Reset index of the new DataFrame
sampled_df = sampled_df.reset_index(drop=True)

In [20]:
# Count the number of packets for each target and print it
target_counts = sampled_df['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown               5000
netatmo_cam           5000
amazon_echo           5000
smart_things          5000
samsung_cam           5000
tp-link_cam           5000
belkin_switch         5000
triby_speaker         5000
belkin_motion         5000
pixstar_photoframe    5000
netatmo_weather       5000
withings_sleep        5000
Name: count, dtype: int64


In [23]:
# List of targets you want to keep
targets_to_keep = [
    'unknown', 'netatmo_cam', 'amazon_echo', 'smart_things', 
    'samsung_cam', 'tp-link_cam', 'belkin_switch', 'triby_speaker', 
    'belkin_motion', 'pixstar_photoframe', 'netatmo_weather'
]

# Filter the DataFrame to keep only the specified targets
filtered_df = sampled_df[sampled_df['target'].isin(targets_to_keep)]

In [24]:
# Count the number of packets for each target and print it
target_counts = filtered_df['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
target
unknown               5000
netatmo_cam           5000
amazon_echo           5000
smart_things          5000
samsung_cam           5000
tp-link_cam           5000
belkin_switch         5000
triby_speaker         5000
belkin_motion         5000
pixstar_photoframe    5000
netatmo_weather       5000
Name: count, dtype: int64


In [ ]:
df_train_test.to_excel('/gdrive/MyDrive/IomT/unsw_data/flow/def_flow_data/df_test_sampled_df_packet_23_28.xlsx', index=False)

In [ ]:
df_holdout.to_excel('/gdrive/MyDrive/IomT/unsw_data/flow/def_flow_data/df_holdout_sampled_df_flow_23_28.xlsx', index=False)